# Simple linear support vector regression to deconvolute single-worm proteomics data

Input format


```
WormbaseID, Group1_rep1, Group1_rep2, Group2_rep1, Group_rep2, ...
WBGene000001, 0, 1, 5, 6, ...
WBGene000002, 10, 11, 3, 2, ...
...
```

Runtime > Run all > click "Choose files" of below cell

In [ ]:
#@title Set parameters and `Runtime` -> `Run all`
from google.colab import files
import os
import re
import hashlib
import random
import glob

from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

#@markdown - Parameters for reference profile
MIN_NUMBER_OF_MARKER = 5 #@param {type:"slider", min:0, max:50, step:1}
MIN_MARKER_SCORE = 0.08 # @param {type:"slider", min:0.0, max:1, step:0.01}
LSVR_C = 0.01 # @param {type:"slider", min:0.0, max:1, step:0.005}
LSVR_E = 0.1 # @param {type:"slider", min:0.0, max:1, step:0.05}
#@markdown - List of interested genes (WBid/ENSEMBLGENE). must be this format "["Gene1", "Gene2"...]"
CELL_REFERENCE_PROFILE = "Abbas-2023" #@param {type:"string"} ["Abbas-2023"]

#@markdown - File format csv-(,) / tsv-(tab)
FILE_FORMAT = "csv" #@param {type: "string"} ["csv", "tsv"]

sep_dict = {'csv': ',', 'tsv': '\t'}
sep = sep_dict[FILE_FORMAT]
uploaded = files.upload()
os.makedirs('./results', exist_ok=True)

# deconvolution with LSVR

In [ ]:
! git clone https://github.com/iron-lion/deconvolution_C_elegans.git

In [ ]:
import sys
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import preprocessing
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
import glob
sys.path.append('./deconvolution_C_elegans/src')
from lsvr import run_multiprocess_deconvolution
from utils import load_cell_to_group, zhu_ref_data
import plotly.express as px

marker_genes_convert = load_cell_to_group(filepath='./deconvolution_C_elegans/data/abbas_cell_markers.csv')

In [ ]:
from itertools import combinations, permutations
import random

def run_one_lsvr(target_df, timepoints, param_C=0.001, param_e=0.1, min_marker=0, min_markerscore=0.05):
    markers = pd.read_csv('./deconvolution_C_elegans/data/top_100_markers_postsub.csv', index_col=None, header=0)
    markers = markers[markers['marker_score'] > min_markerscore]

    bulk_protein_matrix = target_df.sort_index()
    bulk_protein_matrix = bulk_protein_matrix.reindex(set(markers['gene_id']).intersection(target_df.index))
    bulk_protein_matrix = bulk_protein_matrix.dropna(axis=1, how='all').T

    marker_genes = {}
    exp_level = {}
    last_type = ''
    for i, row in markers.iterrows():
        if row['gene_id'] not in bulk_protein_matrix.columns:
            continue

        exp_level[row['gene_id']] = row['mean_expression']
        last_type = row['cell_group']
        if last_type in marker_genes.keys():
            marker_genes[last_type].extend([row['gene_id']])
        else:
            marker_genes[last_type] = [row['gene_id']]


    #markers['weighted_mean_expression'] = markers['marker_score'] * markers['mean_expression']
    # --- Marker genes and existing genes
    reference_df = markers.pivot_table(
            index=['gene_id'], # Use both gene IDs for better indexing
            columns='cell_group',
            values='mean_expression'
        )
    reference_df = reference_df.reindex(bulk_protein_matrix.columns.intersection(reference_df.index))
    reference_df[reference_df.isna()] = 0
    reference_df = reference_df.loc[:, (reference_df>0).sum(axis=0)>min_marker] #################### CUTOFF

    #  Run Deconvolution
    final_proportions_df = run_multiprocess_deconvolution(
        bulk_protein_matrix,
        reference_df,
        max_workers=8 # Use all available cores
    )

    # Concatenate the results into a single DataFrame for final output
    final_df = pd.concat(final_proportions_df, axis=1).T
    final_df.index = bulk_protein_matrix.index

    final_df['timepoints'] = timepoints

    return final_df

In [ ]:
filename = next(iter(uploaded))
target_df = pd.read_csv(filename, sep=sep, header=0, index_col=0)
target_df = target_df[~target_df.index.isna()]

if (sum([True if 'WBGene' in x else False for x in target_df.index]) != target_df.shape[0]):
  raise ValueError('Error: Need WormbaseID')

groups = ['_'.join(x.split('_')[:-1]) for x in target_df.columns]
final_df = run_one_lsvr(target_df, groups, param_C=LSVR_C, param_e=LSVR_E,  min_marker=MIN_NUMBER_OF_MARKER, min_markerscore=MIN_MARKER_SCORE)
final_df.to_csv('./results/proportion.csv')

In [ ]:
final_df = pd.read_csv('./results/proportion.csv', index_col=0, header=0)

# Aggregate
final_df = final_df.T
final_df['group'] = final_df.index
final_df['group'] = final_df['group'].map(marker_genes_convert)
final_df['group'] = [x.split(' is ')[0] if type(x) is str and "These gene" not in x else x for x in final_df['group']]
final_df = final_df.groupby('group').agg('sum')
final_df = final_df.T

final_df['timepoints'] = groups

target_columns = final_df.columns[:-1]
target_columns = final_df.groupby('timepoints').mean().mean().index[final_df.groupby('timepoints').mean().mean() > 0.0]

g = final_df.groupby('timepoints').mean()[target_columns].plot(kind='bar', stacked=True, figsize=(7,4))
plt.legend(loc="upper left", bbox_to_anchor=(1,1))
g.axes.set_ylim((0,1))
g.axes.spines['top'].set_visible(False)
g.axes.spines['right'].set_visible(False)
g.axes.get_xaxis().set_visible(True)
plt.grid(axis='y')

final_df.groupby('timepoints').mean().T.to_csv('./results/plot_data.csv')
plt.savefig('./results/cell_proportion.svg')

In [ ]:
long_df = final_df.groupby('timepoints').mean()[target_columns].reset_index().melt(id_vars='timepoints')
fig = px.bar(long_df, x="timepoints", y="value", color="group", title="Long-Form Input")
fig.write_html("./results/interactive_plot.html")

In [ ]:
!zip -r ./results.zip ./results
from google.colab import files
files.download("./results.zip")